## 0.导入库

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder

## 1.数据集准备

### 1.1 标签处理

通过sklearn的laberlencoder对标签进行处理

In [2]:
labels_name = '1234567890qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'
labels_name = [i for i in labels_name]
encoder = LabelEncoder()
encoder.fit_transform(labels_name)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9,  0, 52, 58, 40, 53, 55, 60, 56,
       44, 50, 51, 36, 54, 39, 41, 42, 43, 45, 46, 47, 61, 59, 38, 57, 37,
       49, 48, 26, 32, 14, 27, 29, 34, 30, 18, 24, 25, 10, 28, 13, 15, 16,
       17, 19, 20, 21, 35, 33, 12, 31, 11, 23, 22], dtype=int64)

### 1.2 图片处理

In [3]:
trans = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

将文件夹中的图片转化为Dataset类， 
1. 如果在初始化的时候传入了transform，则用于测试，输出的值是一个用于测试的列表图片该列表是一个3 30 30 的小图片，5个
2. 如果没有，则是用来训练模型的传出的是一个5 2700的矩阵

In [4]:
# 输出的是一个图片被拆成5个小图片的列表
class captcah(Dataset):
    def __init__(self, root, transform=None):
        self.names = os.listdir('train/train')
        self.images = [os.path.join('train/train', image) for image in self.names]
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.names[index][:-4]
        
        label = [i for i in label]
        label = encoder.transform(label)
        
        image = Image.open(image_path)

        # 如果是外部传入的transform的话，就用于测试传出一个用于测试的列表图片
        # 该列表是一个3 30 30 的小图片，5个
        if self.transform:
            data = self.transform(image)
            list = []
            for i in range(5):
                list.append(data[:,:,i * 30 : (i + 1) * 30].reshape(1, -1))
            return list, label
        
        # else 是内部的，如果没有传入transform，就用默认的， 用来训练模型的
        # 传出的是一个5 2700的矩阵
        else:
            trans = transforms.Compose([
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
            
            data = trans(image)
            list = []
            for i in range(5):
                list.append(data[:,:,i * 30 : (i + 1) * 30].reshape(1, -1))
            
            imgs = list[0]
            for i in range(4):
                imgs = torch.concat((imgs, list[i+1]), axis=0)
            return imgs, label

        
    def __len__(self):
        return len(self.images)

### 1.3 数据集准备

Loader准备

In [5]:
train_dataset = captcah('train/train')
trainLoader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = captcah('train/train', transform=trans)
testLoader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## 2.模型训练

训练模型

In [6]:
torch.__version__

'2.0.1+cpu'

In [7]:
class Cap(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(Cap, self).__init__(*args, **kwargs)
        self.fc1 = nn.Linear(3 * 30 * 30, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 62)

    def forward(self, x):
        x = x.view(-1, 2700)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

cap = Cap()
n_epochs = 10
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cap.parameters(), lr=0.003)
# bestValLost = float('inf')
# earlyStopPatience = 10

lossList = []

for epoch in range(n_epochs):
    for i, data in enumerate(trainLoader):
        images, labels = data
        optimizer.zero_grad()
        output = cap(images)
        loss = criterion(output, labels.reshape(-1,).long())
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'epoch: {epoch + 1}/{n_epochs}, step: {i}/{len(trainLoader)}, loss: {loss.item()}')
            # lossList.append(loss.item())

    # val_loss = loss.item()
    # if val_loss < bestValLost:
    #     bestValLost = val_loss
    #     patience_counters = 0
    # else:
    #     patience_counters += 1
    #     if patience_counters > earlyStopPatience:
    #         print('Early stopping triggered. Training stopped.')
    #         break
    

print('Finish')
torch.save(cap.state_dict(), 'captcah2_0.pth')

epoch: 1/10, step: 0/2000, loss: 4.128220081329346
epoch: 1/10, step: 100/2000, loss: 3.7029056549072266
epoch: 1/10, step: 200/2000, loss: 3.142779588699341
epoch: 1/10, step: 300/2000, loss: 2.5033583641052246
epoch: 1/10, step: 400/2000, loss: 2.329514503479004
epoch: 1/10, step: 500/2000, loss: 1.8909507989883423
epoch: 1/10, step: 600/2000, loss: 1.6405043601989746
epoch: 1/10, step: 700/2000, loss: 1.6601420640945435
epoch: 1/10, step: 800/2000, loss: 1.1999495029449463
epoch: 1/10, step: 900/2000, loss: 2.0748250484466553
epoch: 1/10, step: 1000/2000, loss: 1.6584839820861816
epoch: 1/10, step: 1100/2000, loss: 0.7414701581001282
epoch: 1/10, step: 1200/2000, loss: 1.2972691059112549
epoch: 1/10, step: 1300/2000, loss: 1.4196046590805054
epoch: 1/10, step: 1400/2000, loss: 1.2055482864379883
epoch: 1/10, step: 1500/2000, loss: 0.8174458146095276
epoch: 1/10, step: 1600/2000, loss: 1.2201026678085327
epoch: 1/10, step: 1700/2000, loss: 1.1907281875610352
epoch: 1/10, step: 1800/2

## 3.模型测试

测试判断函数
判断测试传入列表的图片预测值，和所给标签是否相等

In [8]:
def judge(images, labels):
    name = []
    for i in range(len(images)):
        img = images[i].view(1, 2700)
        with torch.no_grad():
            capts = cap(img)
        
        ps = torch.exp(capts)
        probab = list(ps.numpy()[0])
        pre_label = probab.index(max(probab))
        name.append(pre_label)
    
    judgement = (name == list(labels.numpy().reshape(-1,)))
    return judgement

模型准确率评价

In [9]:
all_count, correct_count = 0, 0
cap.eval()
for images, labels in testLoader:
    all_count += 1
    if judge(images, labels):
        correct_count += 1
    if all_count % 100 == 0:
        print(f'Number of Images Tested: {all_count}')
print(f'\n model Accuary:{correct_count / all_count}')

Number of Images Tested: 100
Number of Images Tested: 200
Number of Images Tested: 300
Number of Images Tested: 400
Number of Images Tested: 500
Number of Images Tested: 600
Number of Images Tested: 700
Number of Images Tested: 800
Number of Images Tested: 900
Number of Images Tested: 1000
Number of Images Tested: 1100
Number of Images Tested: 1200
Number of Images Tested: 1300
Number of Images Tested: 1400
Number of Images Tested: 1500
Number of Images Tested: 1600
Number of Images Tested: 1700
Number of Images Tested: 1800
Number of Images Tested: 1900
Number of Images Tested: 2000
Number of Images Tested: 2100
Number of Images Tested: 2200
Number of Images Tested: 2300
Number of Images Tested: 2400
Number of Images Tested: 2500
Number of Images Tested: 2600
Number of Images Tested: 2700
Number of Images Tested: 2800
Number of Images Tested: 2900
Number of Images Tested: 3000
Number of Images Tested: 3100
Number of Images Tested: 3200
Number of Images Tested: 3300
Number of Images Te

In [1]:
import torch

torch.cuda.is_available()

False